In [0]:
dir = '/content/drive/My Drive/DL Project/Data/Train/'

In [2]:
%tensorflow_version 1.x
import keras
import cv2
import numpy as np 
import os

from keras.layers import Dense, LSTM, Flatten
from keras.layers import TimeDistributed, Conv2D, Dropout
from keras import Sequential
from keras.applications.vgg16 import VGG16

TensorFlow 1.x selected.


Using TensorFlow backend.


In [3]:
# we make use of the keras VGG16 "model" for transfer learning
vgg = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

# train only the 4 last layers 
for layer in vgg.layers[:-4]:
    layer.trainable = False

model = Sequential()

# add vgg model for 5 input images
model.add(
    TimeDistributed(vgg, input_shape=(5, 224, 224, 3))
)
model.add(
    TimeDistributed(
        Flatten()
    )
)

model.add(LSTM(256, activation='relu', return_sequences=False))
model.add(Dense(64, activation='relu'))
model.add(Dropout(.5))

#target layer with 3 units
model.add(Dense(3, activation='softmax'))





58892288/58889256 [==============================] - 5s 0us/step







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
optimizer = keras.optimizers.Adam(0.0001)
model.compile(optimizer, loss='categorical_crossentropy', metrics=['acc'])

In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 5, 7, 7, 512)      14714688  
_________________________________________________________________
time_distributed_2 (TimeDist (None, 5, 25088)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               25953280  
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 195       
Total params: 40,684,611
Trainable params: 33,049,347
Non-trainable params: 7,635,264
__________________________________

In [6]:
#find the classes present
from glob import glob
classes = glob(dir+'*')
for i in range(0, len(classes)):
  classes[i] = classes[i].replace(dir,"")

for Class in classes:
  print(Class)

reading book
reading newspaper
not reading


In [0]:
# pattern to get videos and classes
glob_pattern=dir + '{classname}/*.mp4'

In [8]:
import os
import glob
import keras
!pip install keras-video-generators
from keras_video import VideoFrameGenerator

SIZE = (224, 224)
CHANNELS = 3
NBFRAME = 5
BS = 8

# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split=.33, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=True)


  Created wheel for keras-video-generators: filename=keras_video_generators-1.0.13-cp36-none-any.whl size=12729 sha256=f39dabbaa1e6c9e814f0d2e70f887363fb75d9acf45cca0292266dcb7ac91b73
  Stored in directory: /root/.cache/pip/wheels/f4/9a/15/fa18c9caaeb672d7deb2b5be938fcf0c29b259c0cf40aa43f7
Successfully built keras-video-generators


Warning, `split` argument is replaced by `split_val`, please condider to change your source code.The `split` argument will be removed in future releases.


class not reading, validation count: 24, train count: 51
class reading book, validation count: 29, train count: 59
class reading newspaper, validation count: 56, train count: 116
Total data: 3 classes for 226 files for train


In [9]:
valid = train.get_validation_generator()

Total data: 3 classes for 109 files for validation


In [0]:
global history

In [0]:
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        'weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
]


history = model.fit_generator(
    train,
    validation_data=valid,
    verbose=1,
    epochs= 30,
    callbacks=callbacks,
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 6 could not be retrieved. It could be because a worker has died.
  UserWarning)


 7/28 [======>.......................] - ETA: 9:08 - loss: 1.4656 - acc: 0.3929

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 2 could not be retrieved. It could be because a worker has died.
  UserWarning)


 9/28 [========>.....................] - ETA: 8:27 - loss: 1.3841 - acc: 0.4167

...

11/28 [==========>...................] - ETA: 7:54 - loss: 1.3297 - acc: 0.4205

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 25 could not be retrieved. It could be because a worker has died.
  UserWarning)


15/28 [===============>..............] - ETA: 5:51 - loss: 1.2582 - acc: 0.4500

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 13 could not be retrieved. It could be because a worker has died.
  UserWarning)


22/28 [======================>.......] - ETA: 2:34 - loss: 1.1696 - acc: 0.4659

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
print('printking Keys in history')

In [0]:
from keras.utils.vis_utils import plot_model

In [0]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [0]:
from keras.models import load_model

model.save('my_model.h5') 

In [0]:
model.save_weights('my_model_weights.h5')